# HuggingFace_QuickStart_pipeline

파이프라인은 추론을 위해 모델을 사용하는 훌륭하고 쉬운 방법입니다.

이러한 파이프라인은 라이브러리에서 대부분의 복잡한 코드를 추상화하는 개체로, Named Entity Recognition, Masked Language Modeling, 감정 분석, Feature Extraction 및 Question Answering.을 비롯한 여러 task 전용의 간단한 API를 제공합니다.

In [1]:
!pip install -q transformers datasets
!pip install -q sentencepiece
!pip install -q kobert-transformers

In [2]:
from transformers import pipeline

🤗 Transformers 라이브러리 기능을 간단히 살펴보겠습니다.  

라이브러리는 **텍스트의 감정 분석과 같은 자연어 이해(NLU)** 및 **새 텍스트로 프롬프트를 완성하거나 다른 언어로 번역하는 것과 같은 자연어 생성(NLG)** 작업을 위해 사전 훈련된 모델을 다운로드합니다.

먼저 pipeline API를 쉽게 활용하여 추론에서 **사전 훈련된 모델**을 빠르게 사용하는 방법을 살펴보겠습니다. 그런 다음, 라이브러리가 어떻게 이러한 모델에 대한 액세스를 제공하고 **데이터를 사전 처리**하는 데 도움이 되는지 확인 할 것입니다.

## pipeline 으로 작업 시작하기

- 주어진 task 에서 사전 훈련된 모델을 사용하는 가장 쉬운 방법은  `pipeline`을 사용하는 것입니다.

🤗 Transformers 라이브러리는 기본적으로 다음 task를 제공합니다.

- **기계 번역(Translation)**: 다른 언어로 된 텍스트를 번역합니다.  
- **감정 분석(Text Classification)**: 텍스트는 긍정적인가 부정적인가?
- **텍스트 생성(Text Generation)**: 프롬프트를 제공하면 모델이 다음을 생성합니다.
- **이름 개체 인식(NER)**: 입력 문장에서 각 단어를 나타내는 개체(사람, 장소, 등.)
- **질문 답변(Question Answering)**: 모델에 일부 컨텍스트와 질문을 제공하고 컨텍스트에서 답변을 추출합니다.
- **마스킹된 텍스트 채우기(Fill-Mask)**: 마스킹된 단어가 있는 텍스트(예: `[MASK]`로 대체)가 주어지면 공백을 채웁니다.
- **요약(Summarization)**: 긴 텍스트의 요약을 생성합니다.
- **특징 추출(Feature Extraction)**: 텍스트의 텐서 표현을 반환합니다.
- **Zero-Shot 분류(Zero-Shot Classification)**


### pretrained models : https://huggingface.co/models

# Pipeline 사용법
- pipeline('task', model='모델명')

## 기계 번역

- korean pretrained model : https://huggingface.co/Helsinki-NLP/opus-mt-ko-en  

- Helsinki-NLP : University of Helsinki 에서 작성한 다양한 언어 모델 그룹

In [5]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ko-en")

translator("나는 너를 사랑해"), translator("언제나 당신에게 감사함을 느끼고 있습니다.")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


([{'translation_text': 'I love you.'}],
 [{'translation_text': "I'm always grateful to you."}])

## 감정 분석

In [6]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


- 이 명령을 처음 입력하면 사전 훈련된 모델과 해당 토크나이저가 다운로드되어 캐시됩니다.  

- 토크나이저는 모델의 텍스트를 사전 처리한 다음 예측을 담당하는 역할을 합니다.  

- 파이프라인은 모든 것을 함께 그룹화하고 예측을 읽을 수 있도록 사후 처리합니다. 예를 들어:

In [7]:
classifier('We are very happy to show you the 🤗 Transformers library.')

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

사전 처리된 다음 모델에 *batch*로 공급하면 다음과 같은 사전 목록을 반환합니다.

In [8]:
results = classifier([
                      "We are very happy to show you the 🤗 Transformers library.",
                       "We hope you don't hate it."
                        ])
for result in results:
  print(f"label : {result['label']}   with score : {round(result['score'], 4)}")

label : POSITIVE   with score : 0.9998
label : NEGATIVE   with score : 0.5309


## 한국어 감정분석

- NSMC(Naver Sentiment Movie Corpus) 로 미세 조정된 BERT 다국어 basecase 모델 : https://huggingface.co/sangrimlee/bert-base-multilingual-cased-nsmc

In [9]:
classifier_ko = pipeline('sentiment-analysis',
                      model="sangrimlee/bert-base-multilingual-cased-nsmc")

In [10]:
print(classifier_ko("오늘은 정말 즐거운 날이다. 행복하다."))
print(classifier_ko("기분이 꿀꿀해서 술이나 한잔 해야겠다."))
print(classifier_ko('자연어 처리는 어렵다.'))
print(classifier_ko('자연어 처리는 어렵지만 재미있다.'))

[{'label': 'positive', 'score': 0.991436779499054}]
[{'label': 'negative', 'score': 0.8828712105751038}]
[{'label': 'negative', 'score': 0.902804434299469}]
[{'label': 'positive', 'score': 0.9847347736358643}]


In [11]:
classifier_ko([
               "언제나 당신에게 감사함을 느끼고 있습니다.",
               "너한테는 별로 좋은 기억이 없어."
               ])

[{'label': 'positive', 'score': 0.9883792400360107},
 {'label': 'negative', 'score': 0.9782137870788574}]

- 자동 별점 부여

In [12]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment" # 별점 모델
kor_classifier = pipeline('sentiment-analysis', model=model_name)

results = kor_classifier(["다시는 보고 싶지 않은 짜증나는 영화",
                              "아주 재미있는 영화",
                              "정말 재미없는 영화였다",
                              "이 영화 망할거야",
                              "이 영화 최고",
                              "보통 영화"])

for result in results:
    print(f"label: {result['label']}   with score: {round(result['score'], 4)}")

label: 1 star   with score: 0.297
label: 4 stars   with score: 0.3977
label: 2 stars   with score: 0.3608
label: 1 star   with score: 0.2782
label: 5 stars   with score: 0.8198
label: 3 stars   with score: 0.4462


## Zero Shot Pipeline - 처음 보는 문장의 category 분류

In [13]:
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformmers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This is a course about the Transformmers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8423030376434326, 0.11424128711223602, 0.043455611914396286]}

### Zero Shot Pipeline - 다국어 version

In [14]:
classifier = pipeline("zero-shot-classification", model='MoritzLaurer/mDeBERTa-v3-base-mnli-xnli')

sequence = "2023년에 누구에게 투표하겠습니까"
candidate_labels = ["건강", "문화", "정치"]

classifier(sequence, candidate_labels)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


{'sequence': '2023년에 누구에게 투표하겠습니까',
 'labels': ['정치', '문화', '건강'],
 'scores': [0.9428576231002808, 0.032314199954271317, 0.024828176945447922]}

## Text 생성

In [15]:
generator = pipeline("text-generation", model="distilgpt2")
generator(
        "In this course, we will teach you how to",
        max_length=30,
        num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to get started, and a lot more.”'},
 {'generated_text': 'In this course, we will teach you how to read the latest popular Linux programming languages for the first time.'}]

### 한글 Text 생성

- SKT 에서 2021년 5월에 만든 `skt/ko-gpt-trinity-1.2B-v0.5` model 사용 (https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5)

In [3]:
generator = pipeline("text-generation", model="skt/ko-gpt-trinity-1.2B-v0.5")
generator(
        "이 학습 과정에서, 제가 가르치고 싶은 것은",
        max_length=30,
        num_return_sequences=1)

[{'generated_text': "이 학습 과정에서, 제가 가르치고 싶은 것은 '자기주도학습'입니다. 자기주도학습은 학습자가 스스로 학습 목표를 설정하고 학습 계획을 수립하고"}]

In [4]:
generator(
        "스스로 학습 목표를 설정하고 학습 계획을 수립하고",
        max_length=30,
        num_return_sequences=1)

[{'generated_text': '스스로 학습 목표를 설정하고 학습 계획을 수립하고 실천하는 과정을 통해 학습자의 학습 동기를 유발하고 학습자의 학습 의욕을 고취시키는 데 목적이'}]

In [5]:
generator(
        "학습자의 학습 의욕을 고취시키는 데 목적이",
        max_length=30,
        num_return_sequences=1)

[{'generated_text': '학습자의 학습 의욕을 고취시키는 데 목적이 있다. \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n'}]

In [6]:
"이 학습 과정에서, 제가 가르치고 싶은 것은 '자기주도학습'입니다. 자기주도학습은 학습자가" + \
'스스로 학습 목표를 설정하고 학습 계획을 수립하고 실천하는 과정을 통해' + \
'학습자의 학습 의욕을 고취시키는 데 목적이 있다'

"이 학습 과정에서, 제가 가르치고 싶은 것은 '자기주도학습'입니다. 자기주도학습은 학습자가스스로 학습 목표를 설정하고 학습 계획을 수립하고 실천하는 과정을 통해학습자의 학습 의욕을 고취시키는 데 목적이 있다"

### 마스킹된 텍스트 채우기

In [7]:
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.19619806110858917,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052723944187164,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

### 개체명 인식

In [8]:
ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvian and I work at Hugging Face in Brooklyn")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.99876773,
  'word': 'Sylvian',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9672196,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9846444,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

### 질의 응답

In [9]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work ?",
    context = "My name is Sylvian and I work at Hugging Face in Brooklyn"
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.7309892773628235, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}